In [17]:
import pennylane as qml
import matplotlib.pyplot as plt
from pennylane import numpy as np
from scipy.linalg import expm

from math import pi
import numpy as np
import pandas as pd
import random
import torch
from copy import deepcopy

In [18]:
from sklearn import preprocessing
devSet = pd.read_csv("./us_migration.csv")
devSet = devSet.loc[:, ~devSet.columns.str.contains('^Unnamed')]
devSet = devSet.apply(lambda x: pd.to_numeric(x, errors='coerce'))
devSet = devSet.dropna(axis=1)

y = torch.Tensor(devSet['US_MIG_05_10'].values)
X = devSet.loc[:, devSet.columns != "US_MIG_05_10"].values

mMScale = preprocessing.MinMaxScaler()
X = mMScale.fit_transform(X)

x = np.reshape(X[0][0:5], (1, 5))
y = torch.tensor(y.detach().numpy()[0])

print(x)
print(y)

[[0.2898364  0.15086376 0.01904088 0.17205208 0.13949271]]
tensor(961.)


In [19]:
devX = qml.device('default.qubit', wires = len(x[0]))
devY = qml.device('default.qubit', wires = len(x[0]))
devZ = qml.device('default.qubit', wires = len(x[0]))

@qml.qnode(devX)
def rotation_circuitX(vals, thetas):

    # Apply Hadamards
    for hadamard_wire in range(len(vals)):
        qml.Hadamard(wires = hadamard_wire)

    # Apply value dependent Z-axis rotations
    for rotation_val in range(len(vals)):
        qml.RZ(vals[rotation_val], wires = rotation_val)

    # Random CNOT
    qml.CNOT(wires = [1,0])

    # Parametized rotation <- this is what is being trained
    for theta in range(len(thetas)):
        qml.RY(thetas[theta], wires=theta)

    # Get expected values & return them
    expected_values = [qml.expval(qml.PauliX(wire)) for wire in range(len(vals))]
    return expected_values


@qml.qnode(devY)
def rotation_circuitY(vals, thetas):

    # Apply Hadamards
    for hadamard_wire in range(len(vals)):
        qml.Hadamard(wires = hadamard_wire)

    # Apply value dependent Z-axis rotations
    for rotation_val in range(len(vals)):
        qml.RZ(vals[rotation_val], wires = rotation_val)

    # Random CNOT
    qml.CNOT(wires = [1,0])

    # Parametized rotation <- this is what is being trained
    for theta in range(len(thetas)):
        qml.RY(thetas[theta], wires=theta)

    # Get expected values & return them
    expected_values = [qml.expval(qml.PauliY(wire)) for wire in range(len(vals))]
    return expected_values


@qml.qnode(devZ)
def rotation_circuitZ(vals, thetas):

    # Apply Hadamards
    for hadamard_wire in range(len(vals)):
        qml.Hadamard(wires = hadamard_wire)

    # Apply value dependent Z-axis rotations
    for rotation_val in range(len(vals)):
        qml.RZ(vals[rotation_val], wires = rotation_val)

    # Random CNOT
    qml.CNOT(wires = [1,0])

    # Parametized rotation <- this is what is being trained
    for theta in range(len(thetas)):
        qml.RY(thetas[theta], wires=theta)

    # Get expected values & return them
    expected_values = [qml.expval(qml.PauliZ(wire)) for wire in range(len(vals))]
    return expected_values


In [20]:
###### Define our model
class QuantumCicuitNet(torch.nn.Module):
    def __init__(self, n_vals, n_dim):
        super().__init__()
        self.relu = torch.nn.ReLU(inplace=True)
        self.conv2d = torch.nn.Conv2d(1, 1, kernel_size=(3,5), stride=(2,2), padding=(1,1), bias=False)
        self.linear = torch.nn.Linear(4, 1)  


    def param_shift(self, vals, thetas, axis):
        # using the convention u=1/2
        if axis == 'X':
            r_plus = rotation_circuitX(vals, np.array(thetas) + np.array(np.pi / 2))
            r_minus = rotation_circuitX(vals, np.array(thetas) - np.array(np.pi / 2))
            return 10 * (r_plus - r_minus)
        elif axis == 'Y':
            r_plus = rotation_circuitY(vals, np.array(thetas) + np.array(np.pi / 2))
            r_minus = rotation_circuitY(vals, np.array(thetas) - np.array(np.pi / 2))
            return 10 * (r_plus - r_minus)
        elif axis == 'Z':
            r_plus = rotation_circuitZ(vals, np.array(thetas) + np.array(np.pi / 2))
            r_minus = rotation_circuitZ(vals, np.array(thetas) - np.array(np.pi / 2))
            return .5 * (r_plus - r_minus)
        

    def calc_circ_grad(self, x, thetas, axis):
        param_shift_vals = self.param_shift(x, thetas, axis)
        return param_shift_vals

        
    def forward(self, x, thetas):

        outX = torch.tensor(rotation_circuitX(x, thetas), dtype = torch.float32) * 100# OUT:  torch.Size([100, 1, 10, 10])
        outY = torch.tensor(rotation_circuitY(x, thetas), dtype = torch.float32) * 100# OUT:  torch.Size([100, 1, 10, 10])
        outZ = torch.tensor(rotation_circuitZ(x, thetas), dtype = torch.float32) * 100# OUT:  torch.Size([100, 1, 10, 10])
        out = torch.reshape(torch.cat((outX, outY,outZ), 0), (1, 1, 3, 5))

        gradX = torch.tensor(self.calc_circ_grad(x, thetas, 'X'))
        gradY = torch.tensor(self.calc_circ_grad(x, thetas, 'Y'))
        gradZ = torch.tensor(self.calc_circ_grad(x, thetas, 'Z'))
        grads = torch.reshape(torch.cat((gradX, gradY,gradZ), 0), (3,5))
        grads = torch.mean(input = grads, dim = 0)

        out = self.conv2d(out)
        out = self.relu(out)
        out = out.flatten()
        out = self.linear(out)
        return out, grads

In [21]:
lr = 1e-7
model = QuantumCicuitNet(5, 1)
thetas = [.5] * 5

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr = lr)

for i in range(0, 10):

    print("EPOCH: ", i)

    y_pred, grad = model([0.2898364, 0.15086376, 0.01904088, 0.17205208, 0.13949271], thetas)
    loss = criterion(y_pred, y)

    print("    Loss: ", loss)
    print("    Y pred: ", y_pred)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    thetas = [i.item() for i in list(torch.tensor(thetas) + (grad * lr))]

    print("\n")

EPOCH:  0
    Loss:  tensor(928859.9375, grad_fn=<MseLossBackward>)
    Y pred:  tensor([-2.7738], grad_fn=<AddBackward0>)


EPOCH:  1
    Loss:  tensor(927075.3750, grad_fn=<MseLossBackward>)
    Y pred:  tensor([-1.8475], grad_fn=<AddBackward0>)


EPOCH:  2
    Loss:  tensor(925334.4375, grad_fn=<MseLossBackward>)
    Y pred:  tensor([-0.9431], grad_fn=<AddBackward0>)


EPOCH:  3
/Users/heatherbaier/anaconda/envs/qnn/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
    Loss:  tensor(923623.4375, grad_fn=<MseLossBackward>)
    Y pred:  tensor([-0.0533], grad_fn=<AddBackward0>)


EPOCH:  4
    Loss:  tensor(921928.5000, grad_fn=<MseLossBackward>)
    Y pred:  tensor([0.8289], grad_fn=<AddBackward0>)


EPOCH:  5
 